In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Data preprocessing

In [ ]:
train_df = pd.read_csv('../input/santander-customer-satisfaction/train.csv', encoding='latin-1')
test_df = pd.read_csv('../input/santander-customer-satisfaction/test.csv', encoding='latin-1')
print(train_df.shape)
print(test_df.shape)

Total 370 features. Label is named `TARGET`.

In [ ]:
train_df.head()

In [ ]:
train_df.info()

All features are type of int64.

In [ ]:
train_df.isnull().sum()

No missing values.

In [ ]:
train_df['TARGET'].value_counts()

In [ ]:
train_df.describe()

min of var3 is `-999999`, which means NaN or some exception value is replaced by `-999999`.

In [ ]:
train_df['var3'].value_counts()

Replace `-999999` with `2`.

In [ ]:
train_df['var3'] = train_df['var3'].replace(-999999, 2)

Drop ID column.

In [ ]:
train_df = train_df.drop('ID', axis=1)
test_df = test_df.drop('ID', axis=1)

# train_test_split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_df.iloc[:, :-1], train_df.iloc[:, -1], test_size=0.2)

# Model

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2, f_classif, VarianceThreshold
from sklearn.decomposition import PCA

In [ ]:
pipeline = Pipeline([
    ('vt', VarianceThreshold()),
    ('skb', SelectKBest(f_classif, k=15)),
    ('pca', PCA()),
    ('xgb_clf', XGBClassifier())
])

In [ ]:
params = {
    'pca__n_components'         : [4, 5, 6, 7, 8, 9, 10, 11, 12],
    'xgb_clf__learning_rate'    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30],
    'xgb_clf__max_depth'        : [3, 4, 5, 6, 8, 10, 12, 15],
    'xgb_clf__min_child_weight' : [1, 3, 5, 7],
    'xgb_clf__gamma'            : [0.0, 0.1, 0.2 , 0.3, 0.4],
    'xgb_clf__colsample_bytree' : [0.3, 0.4, 0.5 , 0.7]
}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
pipeline_RandomGrid = RandomizedSearchCV(estimator=pipeline, param_distributions=params,
                                         cv=5, n_jobs=-1, scoring='roc_auc', n_iter=5)

In [ ]:
pipeline_RandomGrid.fit(X_train, y_train)

In [ ]:
pipeline_RandomGrid.best_estimator_

In [ ]:
print('Train AUC:', roc_auc_score(y_train, pipeline_RandomGrid.predict_proba(X_train)[:, 1]))
print('Test AUC:', roc_auc_score(y_test, pipeline_RandomGrid.predict_proba(X_test)[:, 1]))

# Feature importance

In [ ]:
from xgboost import plot_importance

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
plot_importance(pipeline_RandomGrid.best_estimator_['xgb_clf'], ax=ax, max_num_features=20, height=0.4)

# Prediction

In [ ]:
X_train = train_df.iloc[:, :-1]
y_train = train_df.iloc[:, -1]
X_test = test_df

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

In [ ]:
preds = pipeline_RandomGrid.predict_proba(X_test)[:, 1]
preds

# Submission

In [ ]:
submission = pd.read_csv('../input/santander-customer-satisfaction/sample_submission.csv', encoding='latin-1')
submission.head()

In [ ]:
submission.loc[:, 'TARGET'] = preds
submission

In [ ]:
submission.to_csv('submission.csv', index=False)